# word2vec implementation

## Overview
<img src="image/word2vec_cbow_mechanism.png" align="left"/>

---
# Setups

In [1]:
import sys
import os
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

## Jupyter notebook setups

In [3]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [4]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio

# Constants

In [5]:
USE_PTB = True
DEBUG = False
VALIDATION = True

---

# Data
## Corpus

In [6]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [7]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Word indexing

In [8]:
%autoreload 2
from layer.preprocessing import (
    WordIndexing, 
    EventContext
)

## WordIndexing instance for the corpus

Adapt to the ```corpus``` and provides:
* word_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [9]:
word_indexing = WordIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)

In [21]:
words = word_indexing.list_words(range(10))
print(f"WordIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_word_indices(words)
print(f"WordIndexing.word_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nWordIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

WordIndexing.vocabulary[10]:
['<nil>' '<unk>' 'develop' 'est' 'request' 'hero' 'breakfast' 'nutritional' 'art' 'satisfaction']

WordIndexing.word_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('develop', 2)
('est', 3)
('request', 4)
('hero', 5)
('breakfast', 6)
('nutritional', 7)
('art', 8)
('satisfaction', 9)

WordIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 5.05607e-02
develop              : 9.09688e-05
est                  : 2.47076e-05
request              : 6.73843e-05
hero                 : 1.01076e-05
breakfast            : 1.12307e-05
nutritional          : 6.73843e-06
art                  : 7.97381e-05
satisfaction         : 2.02153e-05


## Sentence to Sequence

In [11]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :  6072
asbestos        :  9467
fiber           :  2220
<unk>           :     1
is              :  8569
unusually       :  7332
<unk>           :     1
once            :  2480
it              :  6455
enters          :  1875
the             :  6072
<unk>           :     1
with            :  1737
even            :  9532
brief           :  1111
exposures       :  1120
to              :  3007
it              :  6455
causing         :  1069
symptoms        :   738
that            :  1407
show            :  5812
up              :  4301
decades         :  7624
later           :  8536
researchers     :  9612
said            :  1758
